In [10]:
# coding:utf-8

import uuid
import os
import sys
import random


class Partition:

    def __init__(self, lines, tmp_path="tmp/"):
        self.tmp_path = tmp_path
        self.partition = self.write(lines)

    def write(self, lines):
        name_partition = str(uuid.uuid4())
        file_partition = open(self.tmp_path + name_partition, "a")
        file_partition.write("\n".join(lines))
        file_partition.close()
        return self.tmp_path + name_partition

    def map(self, func):
        partitions = []
        partition_data = open(self.partition, 'r')
        lines = []
        for line in partition_data:
            if line.strip():
                lines.append(func(line.strip()))
            if sys.getsizeof(lines) > 1024 * 1024 * 5:
                partitions.append(Partition(lines))
                lines = []

        if len(lines) > 0:
            partitions.append(Partition(lines))

        return partitions

    def filter(self, func):
        partitions = []
        partition_data = open(self.partition, 'r')
        lines = []
        for line in partition_data:
            if func(line.strip()):
                lines.append(line.strip())
            if sys.getsizeof(lines) > 1024 * 1024 * 5:
                partitions.append(Partition(lines))
                lines = []
        if lines:
            partitions.append(Partition(lines))

        return partitions

    def foreach(self, func):
        partition_data = open(self.partition, 'r')
        for line in partition_data:
            func(line)


class RDD:

    def __init__(self, path, tmp_path="tmp/"):
        print('create RDD')

        self.partitions = []

        ls = os.listdir(tmp_path)
        for name_file in ls:
            os.remove(os.path.join(tmp_path, name_file))

        self.tmp_path = tmp_path

        lines = []
        file_data = open(path, 'r')
        for line in file_data:
            lines.append(line)
            if sys.getsizeof(lines) > 1024 * 1024 * 5:
                self.partitions.append(pt.Partition(lines, tmp_path))
                lines = []

        if lines:
            self.partitions.append(Partition(lines, tmp_path))
        file_data.close()

    def copy(self):
        self.partitions = []

    def map(self, func):
        tmp_partitions = self.partitions.copy()
        self.partitions = []
        for partition in tmp_partitions:
            self.partitions.extend(partition.map(func))
            os.remove(partition.partition)
        return self

    def filter(self, func):
        tmp_partitions = self.partitions.copy()
        self.partitions = []
        for partition in tmp_partitions:
            self.partitions.extend(partition.filter(func))
            os.remove(partition.partition)
        return self

    def foreach(self, func):
        for partition in self.partitions:
            partition.foreach(func)
            os.remove(partition.partition)


def map_func(feature):
    vec = feature.split(',')
    vec.append(str(random.uniform(0, 1)))
    return ",".join(vec)


def train_filter_func(value):
    return float(value.split(',')[-1]) < 0.8


def test_filter_func(value):
    return float(value.split(',')[-1]) >= 0.8


def count(value_a, value_b):
    return int(value_a) + int(value_b)


def foreach_func(value):
    print(value.strip())


if __name__ == '__main__':
    rdd = RDD("/home/demo/bigdata-in-action/data/output-feature-csv/part-00000")
    rdd = rdd.map(map_func)
    rdd.filter(test_filter_func).foreach(foreach_func)


create RDD
3596,311,4,44,20141021,2,1,0,1,0.8425709958038121
1,254,1,34,20141021,3,1,0,1,0.8697639251385058
3599,313,0,44,20141021,1,1,0,1,0.8717569421647333
3601,313,0,44,20141021,1,1,0,1,0.9368510727564999
3602,297,1,9,20141021,1,1,0,1,0.8425528120111014
3603,313,0,44,20141021,2,1,0,1,0.8638087529838397
3604,297,1,9,20141021,1,1,0,1,0.9252254198806521
3605,313,0,44,20141028,1,1,1,1,0.8911785506560415
8,67,1,34,20141021,2,1,1,1,0.8830592485732013
3614,313,0,44,20141021,2,1,1,1,0.9877392788003712
3614,313,0,44,20141022,2,1,2,2,0.8218334961304103
3614,313,0,44,20141024,2,1,2,2,0.916147090074016
18,531,1,34,20141027,1,1,2,1,0.8767739204997684
3616,8,4,44,20141021,1,1,2,1,0.99278817680428
3617,313,0,44,20141021,1,1,2,1,0.8913838863085365
3623,465,1,34,20141021,1,1,1,1,0.9611155752112833
3631,612,4,44,20141025,1,1,1,1,0.9353240816072803
3634,611,0,44,20141023,1,1,1,1,0.8845329278567012
3637,313,0,44,20141021,1,1,1,1,0.8617873605318868
32,313,0,44,20141021,1,1,1,1,0.9516868054244288
36,405,

2427,296,1,18,20141021,5,1,20,16,0.8051753943081084
2427,300,1,34,20141021,1,215,20,215,0.9017281121539483
2427,301,1,34,20141022,6,215,3,3,0.8873619582134515
2427,301,1,34,20141024,6,215,3,3,0.8654735483623062
2427,301,1,34,20141026,6,215,3,3,0.8621075521989833
2427,313,1,44,20141022,60249,8,14334,14334,0.8055568226633646
2427,313,1,44,20141024,60249,8,14334,14334,0.9812652095808919
2427,313,1,44,20141028,60249,8,14334,14334,0.883824150776857
2427,330,1,34,20141022,16,3,2,2,0.8678439995285191
2427,331,1,34,20141021,3,3,2,2,0.9805844733787885
2427,331,1,34,20141022,3,3,2,2,0.8598174070488919
2427,700,1,34,20141023,3,79,2,79,0.961951073233116
2427,333,1,45,20141021,2243,79,2,79,0.9123320043340863
2427,333,1,45,20141024,2243,79,124,124,0.8287307564008256
2427,333,1,45,20141026,2243,79,124,124,0.9231978426975402
6031,313,0,44,20141021,4,79,124,124,0.8881125512521533
6031,313,0,44,20141022,4,79,4,4,0.9515792743108331
6032,313,0,44,20141021,1,79,4,4,0.9998541055855751
2434,703,4,44,20141021

In [11]:
# coding:utf-8

import uuid
import os
import sys
import copy


class Partition:

    def __init__(self, partition_data=[], lazy=[], tmp_dir="tmp"):
        self.partition_data = partition_data
        self.lazy = lazy
        self.tmp_dir = tmp_dir
        self.partition_path = []
        self.iterators = []
        self.write()

    def iter(self):
        iter_data = []
        for file in self.partition_path:
            partition_data = open(file, 'r')
            for datum in partition_data:
                iter_data.append(datum.strip())
        return iter_data

    def clear(self):
        self.iterators = []
        self.partition_data = []
        for path in self.partition_path:
            os.remove(path)

    def write(self):
        name_partition = str(uuid.uuid4())
        file_partition = open(os.path.join(self.tmp_dir, name_partition), "a")
        file_partition.write("\n".join(self.partition_data))
        file_partition.close()
        self.partition_path.append(os.path.join(self.tmp_dir, name_partition))
        self.partition_data = []

    def map(self, func):
        reduce_partitions = []
        data = []
        for datum in self.iter():
            if datum:
                data.append(func(datum))
            if sys.getsizeof(data) > 1024 * 1024 * 5:
                reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
                data = []
        if data:
            reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        return reduce_partitions

    def filter(self, func):
        reduce_partitions = []
        data = []
        for datum in self.iter():
            if func(datum):
                data.append(datum)
            if sys.getsizeof(data) > 1024 * 1024 * 5:
                reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        if data:
            reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        return reduce_partitions

    def reduce(self, value, func):
        for datum in self.iter():
            value = func(value, datum)
        self.clear()
        self.partition_data = [value]
        return str(value)

    def action(self):
        if self.lazy:
            op = self.lazy[0]
            self.lazy.remove(op)
            if op[0] is "map":
                return self.map(op[1])
            if op[0] is "filter":
                return self.filter(op[1])
            raise Exception("Unsupported action %s" % op[0])
        self.write()
        return [self]


class RDD:
    MAP = "map"
    FILTER = "filter"
    REDUCE = "reduce"

    def __init__(self, path, tmp_dir="tmp/"):
        print('create RDD')
        self.path = path
        self.tmp_dir = tmp_dir
        self.partitions = []
        self.lazy = []

    def load(self):
        ls = os.listdir(self.tmp_dir)
        for name_file in ls:
            os.remove(os.path.join(self.tmp_dir, name_file))
        reduce_partitions = []
        lines = []
        file_data = open(self.path, 'r')
        for line in file_data:
            lines.append(line.strip())
            if sys.getsizeof(lines) > 1024 * 1024 * 4:
                reduce_partitions.extend(pt.Partition(lines, copy.copy(self.lazy)).action())
                lines = []
        file_data.close()
        if lines:
            reduce_partitions.extend(Partition(lines, copy.copy(self.lazy)).action())
        return reduce_partitions

    def map(self, func):
        self.lazy.append((self.MAP, func))
        return self

    def filter(self, func):
        self.lazy.append((self.FILTER, func))
        return self

    def reduce(self, value, func):
        reduce_partitions = self.load()
        for reduce_partition in reduce_partitions:
            value = reduce_partition.reduce(value, func)
        return value


def map_func(feature):
    return feature.split(',')[-1]


def filter_func(value):
    return value is '0'


def count(value_a, value_b):
    return int(value_a) + 1

if __name__ == '__main__':
    rdd = RDD("/home/demo/bigdata-in-action/data/output-feature-csv/part-00000")
    print("垃圾广告数量为：%s\n" % rdd.map(map_func).filter(filter_func).reduce(0, count))


create RDD
垃圾广告数量为：1



In [12]:
# coding:utf-8
import uuid
import os
import sys
import copy
import pickle


class Partition:

    def __init__(self, partition_data=[], lazy=[], tmp_dir="tmp"):
        self.partition_data = partition_data
        self.lazy = lazy
        self.tmp_dir = tmp_dir
        self.partition_path = []
        self.iterators = []
        self.write()

    def iter(self):
        iter_data = []
        for file in self.partition_path:
            partition_data = pickle.load(open(file, 'rb'))
            for datum in partition_data:
                iter_data.append(datum)
        return iter_data

    def clear(self):
        self.iterators = []
        self.partition_data = []
        for path in self.partition_path:
            os.remove(path)

    def write(self):
        name_partition = str(uuid.uuid4())
        file_partition = open(os.path.join(self.tmp_dir, name_partition), "wb")
        pickle.dump(self.partition_data, file_partition)
        file_partition.close()
        self.partition_path.append(os.path.join(self.tmp_dir, name_partition))
        self.partition_data = []

    def map(self, func):
        reduce_partitions = []
        data = []
        for datum in self.iter():
            if datum:
                data.append(func(datum))
            if sys.getsizeof(data) > 1024:
                reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
                data = []
        if data:
            reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        return reduce_partitions

    def filter(self, func):
        reduce_partitions = []
        data = []
        for datum in self.iter():
            if func(datum):
                data.append(datum)
            if sys.getsizeof(data) > 1024 * 1024 * 5:
                reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        if data:
            reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        return reduce_partitions

    def reduce(self, init_value, func):
        value = init_value
        for datum in self.iter():
            value = func(value, datum)
        self.clear()
        self.partition_data = [value]
        return str(value)

    def reduceByKey(self, init_value, func):
        values = {}
        for datum in self.iter():
            value = values.get(datum[0], init_value)
            value = func(value, datum[1])
            values[datum[0]] = value
        return values

    def action(self):
        if self.lazy:
            op = self.lazy[0]
            self.lazy.remove(op)
            if op[0] is "map":
                return self.map(op[1])
            if op[0] is "filter":
                return self.filter(op[1])
            raise Exception("Unsupported action %s" % op[0])
        self.write()
        return [self]


class RDD:
    MAP = "map"
    FILTER = "filter"
    REDUCE = "reduce"

    def __init__(self, path, tmp_dir="tmp/"):
        print('create RDD')
        self.path = path
        self.tmp_dir = tmp_dir
        self.partitions = []
        self.lazy = []

    def clear(self):
        self.partitions = []

    def load(self):
        ls = os.listdir(self.tmp_dir)
        for name_file in ls:
            os.remove(os.path.join(self.tmp_dir, name_file))
        lines = []
        file_data = open(self.path, 'r')
        for line in file_data:
            lines.append(line.strip())
            if sys.getsizeof(lines) > 128:
                self.partitions.extend(Partition(lines, copy.copy(self.lazy)).action())
                lines = []
        file_data.close()
        if lines:
            self.partitions.extend(Partition(lines, copy.copy(self.lazy)).action())
        return self.partitions

    def map(self, func):
        self.lazy.append((self.MAP, func))
        return self

    def filter(self, func):
        self.lazy.append((self.FILTER, func))
        return self

    def reduce(self, init_value, func):
        self.partitions = self.load()
        value = init_value
        for partition in self.partitions:
            value = partition.reduce(value, func)
        return value

    def reduceByKey(self, init_value, func):
        self.load()
        values = {}
        for partition in self.partitions:
            partition_values = partition.reduceByKey(init_value, func)
            for (key, value) in partition_values.items():
                reduce_value = func(values.get(key, init_value), value)
                values[key] = reduce_value
        self.clear()
        self.partitions = [Partition(sorted(values.items(), reverse=True))]
        return self

    def forEach(self, func):
        for partition in self.partitions:
            for value in partition.iter():
                func(value)
        self.clear()


def map_func(feature):
    return feature.split(',')[0], feature.split(',')[-1]


def filter_func(value):
    return value[1] is '1'


def count(value_a, value_b):
    return int(value_a) + int(value_b)


def func(value):
    print(value)

    
if __name__ == '__main__':
    rdd = RDD("/home/demo/bigdata-in-action/data/output-feature-csv/part-00000")
    rdd.map(map_func).filter(filter_func).reduceByKey(0, count).forEach(func)

create RDD
('984', 1)
('983', 1)
('982', 4)
('979', 4)
('978', 1)
('977', 4)
('974', 4)
('973', 4)
('972', 1)
('971', 4)
('97', 4)
('969', 1)
('961', 1)
('957', 1)
('956', 4)
('954', 4)
('951', 1)
('942', 1)
('941', 1)
('936', 1)
('934', 1)
('930', 1)
('928', 1)
('927', 4)
('925', 1)
('922', 1)
('921', 1)
('920', 1)
('917', 1)
('916', 3)
('914', 1)
('913', 5)
('911', 1)
('909', 1)
('903', 5)
('902', 1)
('9', 1)
('899', 1)
('896', 1)
('892', 4)
('888', 4)
('887', 1)
('883', 3)
('88', 1)
('869', 1)
('867', 1)
('866', 1)
('865', 1)
('855', 4)
('851', 1)
('85', 4)
('842', 4)
('841', 1)
('840', 1)
('83', 1)
('828', 3)
('827', 4)
('821', 1)
('816', 1)
('814', 1)
('811', 1)
('81', 1)
('807', 1)
('804', 4)
('803', 4)
('801', 4)
('800', 4)
('8', 1)
('796', 1)
('795', 1)
('792', 4)
('791', 4)
('789', 1)
('788', 1)
('787', 1)
('785', 1)
('784', 4)
('782', 1)
('781', 4)
('780', 1)
('78', 4)
('779', 4)
('778', 5)
('776', 3)
('775', 1)
('774', 1)
('771', 1)
('770', 4)
('767', 4)
('766', 1)
('764', 1

('4796', 1)
('4794', 1)
('4793', 1)
('4792', 1)
('4791', 1)
('4790', 1)
('479', 1)
('4789', 3)
('4786', 4)
('4785', 1)
('4784', 1)
('4782', 1)
('4781', 1)
('4780', 4)
('478', 4)
('4776', 1)
('4775', 4)
('4774', 4)
('4769', 1)
('4768', 1)
('4767', 1)
('4766', 1)
('4765', 4)
('4764', 1)
('4763', 1)
('4762', 4)
('4760', 1)
('4758', 1)
('4757', 1)
('4756', 4)
('4752', 1)
('4751', 1)
('4750', 4)
('4749', 4)
('4747', 1)
('4745', 1)
('4744', 1)
('4743', 1)
('4736', 9)
('4735', 1)
('4734', 1)
('4732', 1)
('4731', 1)
('4730', 1)
('4729', 1)
('4728', 1)
('4727', 1)
('4726', 1)
('4724', 4)
('4715', 1)
('4714', 1)
('4713', 1)
('4711', 1)
('471', 1)
('4708', 1)
('4707', 1)
('4706', 1)
('4704', 1)
('4703', 1)
('4702', 4)
('4700', 1)
('470', 1)
('4697', 1)
('4696', 1)
('4695', 1)
('4694', 1)
('4692', 1)
('4691', 1)
('4690', 4)
('4689', 1)
('4688', 5)
('4686', 1)
('4683', 1)
('4682', 1)
('4681', 1)
('4679', 1)
('4678', 1)
('4675', 4)
('4674', 1)
('4673', 1)
('4671', 1)
('4670', 1)
('4669', 1)
('4668',

('1401', 4)
('1400', 1)
('1398', 1)
('1397', 1)
('1396', 1)
('1395', 4)
('1392', 1)
('1391', 1)
('139', 1)
('1389', 1)
('1381', 1)
('1380', 1)
('138', 1)
('1379', 1)
('1377', 1)
('1376', 1)
('1375', 1)
('1372', 5)
('1371', 1)
('137', 1)
('1369', 4)
('1368', 1)
('1367', 4)
('1365', 5)
('1352', 3)
('1350', 4)
('135', 5)
('1349', 1)
('1347', 3)
('134', 4)
('1339', 1)
('1337', 3)
('1336', 1)
('1335', 4)
('1334', 4)
('1331', 4)
('1330', 1)
('1326', 4)
('1323', 1)
('1320', 1)
('1319', 1)
('1318', 4)
('1316', 1)
('1314', 4)
('1309', 4)
('1306', 1)
('1304', 1)
('1303', 1)
('1302', 1)
('1301', 1)
('1294', 1)
('1293', 1)
('1292', 4)
('1280', 1)
('1275', 1)
('1272', 1)
('1271', 1)
('1269', 1)
('1268', 1)
('1261', 1)
('1260', 5)
('1257', 1)
('1256', 4)
('1254', 1)
('1253', 1)
('1252', 4)
('1241', 1)
('1239', 4)
('1236', 1)
('1233', 4)
('1232', 5)
('1229', 1)
('1228', 4)
('1222', 4)
('1220', 4)
('122', 4)
('1219', 1)
('1218', 4)
('1217', 4)
('1216', 1)
('1213', 1)
('1212', 3)
('1204', 1)
('1203', 3

In [15]:
# coding:utf-8
import uuid
import os
import sys
import copy
import pickle


class Partition:

    def __init__(self, partition_data=[], lazy=[], tmp_dir="tmp"):
        self.partition_data = partition_data
        self.lazy = lazy
        self.tmp_dir = tmp_dir
        self.partition_path = []
        self.iterators = []
        self.write()

    def iter(self):
        iter_data = []
        for file in self.partition_path:
            partition_data = pickle.load(open(file, 'rb'))
            for datum in partition_data:
                iter_data.append(datum)
        return iter_data

    def clear(self):
        self.iterators = []
        self.partition_data = []
        for path in self.partition_path:
            os.remove(path)

    def write(self):
        name_partition = str(uuid.uuid4())
        file_partition = open(os.path.join(self.tmp_dir, name_partition), "wb")
        pickle.dump(self.partition_data, file_partition)
        file_partition.close()
        self.partition_path.append(os.path.join(self.tmp_dir, name_partition))
        self.partition_data = []

    def map(self, func):
        reduce_partitions = []
        data = []
        for datum in self.iter():
            if datum:
                data.append(func(datum))
            if sys.getsizeof(data) > 1024:
                reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
                data = []
        if data:
            reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        return reduce_partitions

    def filter(self, func):
        reduce_partitions = []
        data = []
        for datum in self.iter():
            if func(datum):
                data.append(datum)
            if sys.getsizeof(data) > 1024 * 1024 * 5:
                reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        if data:
            reduce_partitions.extend(Partition(data, copy.copy(self.lazy)).action())
        return reduce_partitions

    def reduce(self, init_value, func):
        value = init_value
        for datum in self.iter():
            value = func(value, datum)
        self.clear()
        self.partition_data = [value]
        return str(value)

    def shuffle(self):
        values = {}
        for datum in self.iter():
            file = values.get(datum[0], [])
            file.append(datum)
            values[datum[0]] = file
        return values

    def action(self):
        if self.lazy:
            op = self.lazy[0]
            self.lazy.remove(op)
            if op[0] is "map":
                return self.map(op[1])
            if op[0] is "filter":
                return self.filter(op[1])
            raise Exception("Unsupported action %s" % op[0])
        self.write()
        return [self]


class RDD:
    MAP = "map"
    FILTER = "filter"
    REDUCE = "reduce"

    def __init__(self, path, tmp_dir="tmp/"):
        print('create RDD')
        self.path = path
        self.tmp_dir = tmp_dir
        self.partitions = []
        self.lazy = []

    def clear(self):
        self.partitions = []

    def load(self):
        ls = os.listdir(self.tmp_dir)
        for name_file in ls:
            os.remove(os.path.join(self.tmp_dir, name_file))
        lines = []
        file_data = open(self.path, 'r')
        for line in file_data:
            lines.append(line.strip())
            if sys.getsizeof(lines) > 128:
                self.partitions.extend(Partition(lines, copy.copy(self.lazy)).action())
                lines = []
        file_data.close()
        if lines:
            self.partitions.extend(Partition(lines, copy.copy(self.lazy)).action())
        return self.partitions

    def map(self, func):
        self.lazy.append((self.MAP, func))
        return self

    def filter(self, func):
        self.lazy.append((self.FILTER, func))
        return self

    def reduce(self, init_value, func):
        self.partitions = self.load()
        value = init_value
        for partition in self.partitions:
            value = partition.reduce(value, func)
        return value

    def reduceByKey(self, init_value, func):
        self.load()
        values = {}
        for partition in self.partitions:
            partition_files = partition.shuffle()
            for (key, files) in partition_files.items():
                for datum in files:
                    reduce_value = func(values.get(key, init_value), datum[1])
                    values[key] = reduce_value
        self.clear()
        self.partitions = [Partition(sorted(values.items(), reverse=True))]
        return self

    def forEach(self, func):
        for partition in self.partitions:
            for value in partition.iter():
                func(value)
        self.clear()


def map_func(feature):
    return feature.split(',')[0], feature.split(',')[-1]


def filter_func(value):
    return value[1] is '1'


def count(value_a, value_b):
    return int(value_a) + int(value_b)


def func(value):
    print(value)


if __name__ == '__main__':
    rdd = RDD("/home/demo/bigdata-in-action/data/output-feature-csv/part-00000")
    rdd.map(map_func).filter(filter_func).reduceByKey(0, count).forEach(func)

create RDD
('984', 1)
('983', 1)
('982', 4)
('979', 4)
('978', 1)
('977', 4)
('974', 4)
('973', 4)
('972', 1)
('971', 4)
('97', 4)
('969', 1)
('961', 1)
('957', 1)
('956', 4)
('954', 4)
('951', 1)
('942', 1)
('941', 1)
('936', 1)
('934', 1)
('930', 1)
('928', 1)
('927', 4)
('925', 1)
('922', 1)
('921', 1)
('920', 1)
('917', 1)
('916', 3)
('914', 1)
('913', 5)
('911', 1)
('909', 1)
('903', 5)
('902', 1)
('9', 1)
('899', 1)
('896', 1)
('892', 4)
('888', 4)
('887', 1)
('883', 3)
('88', 1)
('869', 1)
('867', 1)
('866', 1)
('865', 1)
('855', 4)
('851', 1)
('85', 4)
('842', 4)
('841', 1)
('840', 1)
('83', 1)
('828', 3)
('827', 4)
('821', 1)
('816', 1)
('814', 1)
('811', 1)
('81', 1)
('807', 1)
('804', 4)
('803', 4)
('801', 4)
('800', 4)
('8', 1)
('796', 1)
('795', 1)
('792', 4)
('791', 4)
('789', 1)
('788', 1)
('787', 1)
('785', 1)
('784', 4)
('782', 1)
('781', 4)
('780', 1)
('78', 4)
('779', 4)
('778', 5)
('776', 3)
('775', 1)
('774', 1)
('771', 1)
('770', 4)
('767', 4)
('766', 1)
('764', 1

('3960', 4)
('396', 1)
('3954', 4)
('3952', 4)
('3951', 1)
('395', 4)
('3949', 1)
('3948', 1)
('3947', 1)
('3945', 4)
('3942', 5)
('3941', 4)
('3940', 1)
('394', 1)
('3938', 1)
('3937', 4)
('3933', 1)
('3932', 4)
('3931', 4)
('3930', 1)
('3929', 1)
('3928', 1)
('3927', 4)
('3925', 1)
('3924', 1)
('3923', 1)
('392', 1)
('3918', 1)
('3915', 1)
('3914', 1)
('3913', 1)
('3912', 1)
('3910', 1)
('391', 4)
('3909', 1)
('3908', 1)
('3907', 1)
('3906', 4)
('3905', 1)
('3904', 1)
('3903', 1)
('3901', 1)
('3900', 1)
('390', 1)
('3897', 1)
('3896', 1)
('3894', 1)
('3892', 3)
('3890', 1)
('389', 1)
('3889', 5)
('3888', 1)
('3887', 1)
('3886', 1)
('3884', 1)
('3881', 1)
('3880', 1)
('3879', 1)
('3878', 4)
('3877', 4)
('3876', 1)
('3875', 1)
('3873', 1)
('3872', 1)
('3871', 1)
('3869', 1)
('3865', 1)
('3863', 1)
('3862', 1)
('3861', 1)
('3860', 1)
('3859', 4)
('3858', 1)
('3857', 5)
('3856', 5)
('3855', 1)
('3854', 1)
('3852', 1)
('3851', 3)
('3849', 4)
('3848', 1)
('3847', 1)
('3846', 1)
('3845', 5)